In [39]:
import numpy as np
import pandas as pd
from sklearn.preprocessing import StandardScaler
from sklearn.svm import SVC
from sklearn.model_selection import GridSearchCV,RandomizedSearchCV,train_test_split,StratifiedKFold,cross_val_score
from sklearn.metrics import accuracy_score,roc_auc_score,f1_score,confusion_matrix,recall_score

In [24]:
df = pd.read_csv('train.csv')

In [25]:
df.shape

(42000, 785)

In [26]:
df.head()

,label,pixel0,pixel1,pixel2,pixel3,pixel4,pixel5,pixel6,pixel7,pixel8,...,pixel774,pixel775,pixel776,pixel777,pixel778,pixel779,pixel780,pixel781,pixel782,pixel783
0,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,4,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [27]:
X = df.drop(columns=['label'],axis = 1)

In [28]:
Y = df['label']

In [29]:
X.columns

Index(['pixel0', 'pixel1', 'pixel2', 'pixel3', 'pixel4', 'pixel5', 'pixel6',
       'pixel7', 'pixel8', 'pixel9',
       ...
       'pixel774', 'pixel775', 'pixel776', 'pixel777', 'pixel778', 'pixel779',
       'pixel780', 'pixel781', 'pixel782', 'pixel783'],
      dtype='object', length=784)

In [30]:
Y

0        1
1        0
2        1
3        4
4        0
        ..
41995    0
41996    1
41997    7
41998    6
41999    9
Name: label, Length: 42000, dtype: int64

In [31]:
X_train,X_test,Y_train,Y_test = train_test_split(X,Y, test_size= 0.2 ,random_state=45)

In [32]:
scaler = StandardScaler()

In [35]:
X_trained = scaler.fit_transform(X_train)
X_tested = scaler.transform(X_test)

In [ ]:
model = SVC(class_weight='balanced')
grad = {
    'C':[0.1,0.3,0.5,0.9],
    'kernel':['rbf']
}
grid = RandomizedSearchCV(estimator=model , param_distributions=grad, cv = 3 , error_score='raise',n_jobs = -1 , n_iter= 4)
grid.fit(X_trained,Y_train)


RandomizedSearchCV(cv=3, error_score='raise',
                   estimator=SVC(class_weight='balanced'), n_iter=4, n_jobs=-1,
                   param_distributions={'C': [0.1, 0.3, 0.5, 0.9],
                                        'kernel': ['rbf']})

In [36]:
model = grid.best_estimator_

In [37]:
y_pred = model.predict(X_tested)

In [38]:
accuracy_score(y_pred,Y_test)

0.9563095238095238

In [41]:
cv = StratifiedKFold(n_splits=10)
cross_val_score(model,X,Y,cv = cv , n_jobs = -1 )

array([0.97809524, 0.97452381, 0.98095238, 0.9747619 , 0.97547619,
       0.9702381 , 0.97333333, 0.97952381, 0.98047619, 0.9747619 ])

In [43]:
df2 = pd.read_csv('test.csv')

In [44]:
X = df2

In [45]:
X.columns

Index(['pixel0', 'pixel1', 'pixel2', 'pixel3', 'pixel4', 'pixel5', 'pixel6',
       'pixel7', 'pixel8', 'pixel9',
       ...
       'pixel774', 'pixel775', 'pixel776', 'pixel777', 'pixel778', 'pixel779',
       'pixel780', 'pixel781', 'pixel782', 'pixel783'],
      dtype='object', length=784)

In [46]:
X_ul = scaler.fit_transform(X)

In [47]:
y_last = model.predict(X_ul)

In [50]:
lu = pd.DataFrame({'ImageId':range(1,28001),'Label':y_last})

In [51]:
las = lu.to_csv('final.csv',index = False)